# Surgeon Performance Analysis

Gemma Analytics - Analytics Engineer Task 

# Introduction

This project explores a real-world healthcare dataset to answer a high-stakes question:

> Who are the most — and least — skillful surgeons for hip replacement operations?

To approach this, I analyzed EQ-5D-5L survey responses, a standardized health questionnaire that patients completed **before and after surgery**. Each survey asked patients to rate their condition across five key dimensions:

- **Mobility** (Can the patient walk or move around?)
- **Self-care** (Can they wash or dress themselves?)
- **Usual activities** (Are they able to carry out everyday tasks like work, study, or leisure?)
- **Pain / Discomfort** ( Are they experiencing physical pain or discomfort?)
- **Anxiety / Depression** (How are they feeling mentally or emotionally?)

By comparing these responses pre- and post-surgery, we get a sense of how much each patient improved or didn’t.

From there, the goal is to evaluate which surgeons consistently deliver strong outcomes. This type of outcome-based analysis is incredibly valuable in clinical settings, it focuses not on what was done, but on what actually changed for the patient.


# Task Understanding

The main objective is to identify which surgeons are the most and least skillful when it comes to performing **hip replacement surgeries** based on how much their patients improve after surgery.

To do that, I needed to:

1. **Measure improvement** for each patient  
   → by comparing pre- and post-surgery EQ-5D-5L responses.

2. **Convert responses to numeric scores**  
   → using the provided scoring table (central estimate per answer).

3. **Calculate health score changes**  
   → for each patient, and then group those by surgeon.

4. **Evaluate surgeon performance**  
   → not just by average improvement, but by also considering:
   - How many patients they treated (experience)
   - How consistent their outcomes were (stability)

The final goal is to surface insights that are fair, data-driven, and meaningful, the kind you could confidently share with a medical team, operations manager, or executive stakeholder.


# About the Dataset

The dataset consists of six interconnected tables that reflect patient responses, survey metadata, and surgeon information. Here's a quick overview:

- **`answers`**  
  Contains every individual answer from a patient to a survey question.

- **`answer_options`**  
  Maps each answer text (e.g., "I have moderate pain") to a numerical score (`central_estimate`) used for health scoring.

- **`patients`**  
  Links each patient to a surgeon and includes basic demographic info.

- **`questionnaires`**  
  Indicates whether a set of answers was given **before** or **after** surgery, and whether it was a **hip** or **knee** procedure.

- **`questions`**  
  Describes the 5 EQ-5D-5L dimensions (mobility, pain, etc.).

- **`surgeons`**  
  Provides the names of each surgeon, tied to patient outcomes via `surgeon_id`.

Together, these tables allow us to measure how much each patient’s health changed after surgery and link that change back to the surgeon who performed the procedure.

# Data Loading

I started by loading all six datasets from CSV files into Pandas DataFrames. Each file corresponds to one of the tables described above.

I also created a quick audit function to check:
- Number of rows and columns
- Missing values
- Duplicate rows
- Data types

This gives a clear picture of the structure and cleanliness of the data right from the start.


### Step 1: Loading csv data into a pandas dataframe

In [66]:
import pandas as pd
import os

# Define the data folder
data_path = "data"

# Load all CSVs
answer_options = pd.read_csv(os.path.join(data_path, "answer_options.csv"))
answers = pd.read_csv(os.path.join(data_path, "answers.csv"))
patients = pd.read_csv(os.path.join(data_path, "patients.csv"))
questionnaires = pd.read_csv(os.path.join(data_path, "questionnaires.csv"))
questions = pd.read_csv(os.path.join(data_path, "questions.csv"))
surgeons = pd.read_csv(os.path.join(data_path, "surgeons.csv"))

print("All datasets loaded successfully.")

All datasets loaded successfully.


In [67]:
answer_options.head(5)

,question_id,answer,severity_code,central_estimate
0,1,I have no problems in walking around,1,0.000
1,1,I have slight problems in walking around,2,0.058
2,1,I have moderate problems in walking around,3,0.076
3,1,I have severe problems in walking around,4,0.210
4,1,I am unable to walk around,5,0.270


In [68]:
answers.head(5)

,id,question_id,patient_id,questionnaire_id,answer
0,1,1,1,1,I have moderate problems in walking around
1,2,2,1,1,I have slight problems washing or dressing myself
2,3,3,1,1,I have moderate problems doing my usual activi...
3,4,4,1,1,I have severe pain or discomfort
4,5,5,1,1,I am moderately anxious or depressed


In [69]:
patients.head(5)

,id,gender,surgeon_id
0,1,Male,3
1,2,Male,3
2,3,Female,1
3,4,Female,9
4,5,Male,5


In [70]:
questionnaires.head(5)

,id,type,treatment,questions
0,1,pre,Hip,"[1, 2, 3, 4, 5]"
1,2,post,Hip,"[1, 2, 3, 4, 5]"
2,3,pre,Knee,"[1, 2, 3, 4, 5]"
3,4,post,Knee,"[1, 2, 3, 4, 5]"


In [71]:
questions.head(5)

,id,title,description
0,1,Mobility,Please indicate what applies
1,2,Self-care,Please indicate what applies
2,3,Usual activities,Please indicate what applies
3,4,Pain / discomfort,Please indicate what applies
4,5,Anxiety / depression,Please indicate what applies


In [72]:
surgeons.head(5)

,id,name
0,1,Princess Leia
1,2,Darth Maul
2,3,Yoda
3,4,Boba Fett
4,5,Obi-Wan Kenobi


Each CSV file was loaded into a separate DataFrame using the **Pandas** library in Python. After loading, I displayed the first 5 rows of each dataset using `.head()` to get an initial sense of the structure and content.

This quick preview helps verify that:
- The data loaded correctly
- Column names are as expected
- The format looks consistent across tables

This also sets up the next step, where I will check for data quality issues like missing values, incorrect types, or duplicates.


### Step 2: Data Audit – Checking Structure, Nulls, and Duplicates

Before jumping into analysis or merging data, it’s important to take a moment to audit the raw datasets. This helps confirm that:

- There are no missing values in critical columns
- There are no accidental duplicate rows
- Data types are appropriate for what we want to do next (e.g., IDs as strings or integers)

To make this quick and consistent, I created a simple audit function called `audit_dataframe()`.

This function prints out:
- The shape of the dataset (rows × columns)
- A count of missing values in each column
- The number of duplicated rows (if any)
- A full `.info()` summary including data types and memory usage

This check gives me a full view of data cleanliness and readiness, and helps prevent surprises later when joining or filtering data.


In [73]:
def audit_dataframe(df, name):
    print(f"\n {name}")
    print("-" * 50)
    print(f"Shape: {df.shape}")
    print(f"Missing values:\n{df.isnull().sum()}")
    print(f"Duplicate rows: {df.duplicated().sum()}")
    print("\nData types and memory usage:")
    print(df.info())
    print("-" * 50)

# Run audit on each dataset
audit_dataframe(answer_options, "Answer Options")
audit_dataframe(answers, "Answers")
audit_dataframe(patients, "Patients")
audit_dataframe(questionnaires, "Questionnaires")
audit_dataframe(questions, "Questions")
audit_dataframe(surgeons, "Surgeons")



 Answer Options
--------------------------------------------------
Shape: (25, 4)
Missing values:
question_id         0
answer              0
severity_code       0
central_estimate    0
dtype: int64
Duplicate rows: 0

Data types and memory usage:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question_id       25 non-null     int64  
 1   answer            25 non-null     object 
 2   severity_code     25 non-null     int64  
 3   central_estimate  25 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 932.0+ bytes
None
--------------------------------------------------

 Answers
--------------------------------------------------
Shape: (218335, 5)
Missing values:
id                  0
question_id         0
patient_id          0
questionnaire_id    0
answer              0
dtype: int64
Duplicate rows: 0

Data ty

I used a custom `audit_dataframe()` function to check each table’s shape, missing values (`.isnull().sum()`), duplicate rows (`.duplicated().sum()`), and data types (`.info()`).

Result: 
- No missing or duplicate rows
- Data types are appropriate
- Column names are clear

Next, I’ll move into **data preparation**, where I will normalize key text fields and filter the dataset to include only **hip replacement surgeries**, as required by the task.

# Data Cleaning & Preparation

Although our audit showed no missing values or duplicate rows, it's still good practice to clean key text columns to avoid inconsistencies when joining or filtering.

Specifically, I cleaned:
- `answer` in both `answers` and `answer_options` to match exact text (e.g., removing spaces, fixing casing)
- `treatment` in `questionnaires` to ensure we correctly filter to "Hip" replacements later on

This helps ensure our joins will work correctly and we won’t accidentally exclude rows due to small text mismatches (like "hip" vs. "Hip").


### Step 3:  Standardize Text Columns

In [74]:
# Normalize answer text (remove leading/trailing spaces, capitalize)
answer_options['answer'] = answer_options['answer'].str.strip().str.title()
answers['answer'] = answers['answer'].str.strip().str.title()

# Normalize treatment types for filtering later (e.g., "Hip", "Knee")
questionnaires['treatment'] = questionnaires['treatment'].str.strip().str.title()

# Confirm results
print("Unique treatment types:", questionnaires['treatment'].unique())
print("Sample cleaned answers:", answer_options['answer'].unique()[:5])


Unique treatment types: ['Hip' 'Knee']
Sample cleaned answers: ['I Have No Problems In Walking Around'
 'I Have Slight Problems In Walking Around'
 'I Have Moderate Problems In Walking Around'
 'I Have Severe Problems In Walking Around' 'I Am Unable To Walk Around']


Results: 

- The `treatment` column now contains standardized values: `'Hip'` and `'Knee'`. This confirms the text normalization worked and prepares the data for accurate filtering.
- The `answer` values in both `answers` and `answer_options` have also been cleaned and capitalized consistently. This ensures they'll match correctly when we join these tables later.

Everything looks good and is ready for filtering the dataset to include **only hip replacement surgeries**, as required by the task.


### Step 4: Filter to Only Hip Replacement Surgeries

The original question focuses specifically on **hip replacement operations**, but our dataset includes responses for both hip and knee surgeries.

To make sure our analysis is aligned, we first filter the `questionnaires` table to only include rows where the `treatment` is "Hip".

We’ll then use this filtered version of the table (`hip_questionnaires`) in our later joins, so only hip surgery responses are included throughout the analysis.

In [75]:
# Filter questionnaires to only include Hip surgeries
hip_questionnaires = questionnaires[questionnaires['treatment'] == 'Hip'].copy()

# Confirm results
print(" Hip-only questionnaires retained:", hip_questionnaires.shape[0])
print("Questionnaire types:", hip_questionnaires['type'].unique())


 Hip-only questionnaires retained: 2
Questionnaire types: ['pre' 'post']


While I used Pandas to filter the `questionnaires` table to include only `"Hip"` treatments, I also wanted to demonstrate how the same logic would look and run in SQL.

Using the `pandasql` library, we can run actual SQL queries on our in-memory Pandas DataFrames. This is helpful for:

- Demonstrating SQL fluency
- Aligning with dbt-style workflows (which Gemma uses)
- Validating that the logic produces the same result in both environments

Below, I execute the same filtering logic using SQL to retrieve only hip replacement questionnaire entries.

In [76]:
import pandasql as psql

# Example: select all hip questionnaires
query = """
SELECT *
FROM questionnaires
WHERE treatment = 'Hip'
"""

hip_questionnaires_sql = psql.sqldf(query, globals())
hip_questionnaires_sql.head()

,id,type,treatment,questions
0,1,pre,Hip,"[1, 2, 3, 4, 5]"
1,2,post,Hip,"[1, 2, 3, 4, 5]"


This shows that:
- Only the **pre- and post-surgery EQ-5D-5L questionnaires** for hip replacements are retained.
- Each includes the same 5 questions (IDs 1–5), which will be used to calculate health scores.
- Any knee-related surveys have been excluded, ensuring the analysis stays focused on hip surgeries only.

Now that this filter is in place, I will merge this data with patient responses so that only **hip-related answers** move forward into the analysis. 

### Step 5: Merge Tables for Scoring

This below SQL query combines four key tables to build the dataset required for scoring patient outcomes:

| Table                | Why It’s Needed                       |
| -------------------- | ------------------------------------- |
| `answers`            | Patient answers to EQ-5D-5L questions |
| `answer_options`     | Maps each answer to a numerical score |
| `hip_questionnaires` | Filters to **hip** surgeries only     |
| `patients`           | Links each patient to a surgeon       |


This will give us a table where each row is one scored answer, labeled as pre/post, and tagged with a surgeon_id.

All further logic uses SQL (`pandasql`) for transformation to reflect production-ready dbt-style workflows.


In [77]:
query = """
SELECT 
    a.question_id,
    a.patient_id,
    a.questionnaire_id,
    a.answer,
    ao.central_estimate AS score,
    q.type AS questionnaire_type,
    p.surgeon_id
FROM answers a
JOIN answer_options ao
    ON a.question_id = ao.question_id AND a.answer = ao.answer
JOIN hip_questionnaires q
    ON a.questionnaire_id = q.id
JOIN patients p
    ON a.patient_id = p.id
"""

# Run SQL and store result
scored_answers_sql = psql.sqldf(query, globals())

# Preview the merged table
scored_answers_sql.head()


,question_id,patient_id,questionnaire_id,answer,score,questionnaire_type,surgeon_id
0,1,1,1,I Have Moderate Problems In Walking Around,0.076,pre,3
1,2,1,1,I Have Slight Problems Washing Or Dressing Myself,0.050,pre,3
2,3,1,1,I Have Moderate Problems Doing My Usual Activi...,0.063,pre,3
3,4,1,1,I Have Severe Pain Or Discomfort,0.280,pre,3
4,5,1,1,I Am Moderately Anxious Or Depressed,0.100,pre,3


The result is a flat table where each row contains:
- The question answered
- The patient and questionnaire ID
- The text response and its numeric score
- Whether it was a **pre** or **post** surgery response
- The **surgeon_id** for attribution

This structure sets us up to calculate **pre- and post-surgery scores per patient**, and later aggregate those improvements by surgeon.

### Step 6: Calculate Health Scores and Improvement (SQL via pandasql)

Each patient answered 5 questions before and after surgery. To calculate their health score:

- I group the responses by `patient_id` and `questionnaire_type` (`pre` or `post`)
- I sum the 5 numeric scores (from `answer_options`)
- I convert this to a health score using the formula: `health_score = 1 - SUM(score)`

This formula turns the sum of severity scores into a more intuitive health score — where **higher = better condition**. Each patient should now have two rows: one for their pre-surgery score, and one for post-surgery.

In [78]:
# First, create a temp view to aggregate scores per patient per type
query_scores = """
SELECT
    patient_id,
    questionnaire_type,
    1 - SUM(score) AS health_score
FROM scored_answers_sql
GROUP BY patient_id, questionnaire_type
"""

patient_scores_sql = psql.sqldf(query_scores, globals())
patient_scores_sql.head()


,patient_id,questionnaire_type,health_score
0,1,post,0.690
1,1,pre,0.431
2,"1,001",post,0.500
3,"1,001",pre,0.526
4,"1,006",post,0.795


Example Output:

| patient_id | questionnaire_type | health_score |
|------------|--------------------|---------------|
| 1          | post               | 0.690         |
| 1          | pre                | 0.431         |

This shows that patient 1 improved from 0.431 to 0.690 which is a clear positive change.

---

Why This Matters:

This step gives me exactly what I need to:
- Compare each patient’s condition **before and after surgery**
- Quantify their individual **improvement**
- Attribute that improvement to a surgeon in the next step

Once I have pre- and post-surgery scores, I pivot them into separate columns, and calculate improvement as:

> improvement = post_score - pre_score

This gives us a table where each row represents one patient and their improvement.


#### Step 7: Pivot into Pre/Post Columns (SQL-style via Pandas for now)


Now that I have health scores for each patient both **pre-** and **post-surgery**, I pivoted the table so each patient has a single row with both scores side by side.

This makes it easier to compare and calculate improvement:

improvement = post_score - pre_score

**Note:** Unfortunately, pandasql doesn't support pivot directly (SQL PIVOT is not implemented in SQLite, which pandasql uses). So here we switch briefly to Pandas to perform the pivot step.

In [79]:
# Pivot health scores to have pre and post on the same row
patient_scores_pivot = patient_scores_sql.pivot(
    index='patient_id',
    columns='questionnaire_type',
    values='health_score'
).reset_index()

# Rename columns
patient_scores_pivot.columns.name = None
patient_scores_pivot = patient_scores_pivot.rename(columns={
    'pre': 'pre_score',
    'post': 'post_score'
})

# Calculate improvement
patient_scores_pivot['improvement'] = (
    patient_scores_pivot['post_score'] - patient_scores_pivot['pre_score']
)

# Preview
patient_scores_pivot.head()

,patient_id,post_score,pre_score,improvement
0,1,0.690,0.431,0.259
1,"1,001",0.500,0.526,-0.026
2,"1,006",0.795,0.824,-0.029
3,"1,007",0.510,0.824,-0.314
4,"1,009",0.862,0.744,0.118


Result: 
- Positive values = the patient’s condition improved  
- Negative values = the patient felt **worse** after surgery (a flag worth tracking)


Why This Step Is Important

- This gives a **quantitative measure of recovery** for each patient
- It's the foundation for evaluating **surgeon performance**
- In the next step, I will merge this with surgeon IDs and begin scoring at the surgeon level


In [80]:
# Merge surgeon_id (from scored_answers_sql, dropping duplicates)
patient_surgeons = scored_answers_sql[['patient_id', 'surgeon_id']].drop_duplicates()

patient_scores_pivot = patient_scores_pivot.merge(
    patient_surgeons,
    on='patient_id',
    how='left'
)

patient_scores_pivot.head()


,patient_id,post_score,pre_score,improvement,surgeon_id
0,1,0.690,0.431,0.259,3
1,"1,001",0.500,0.526,-0.026,4
2,"1,006",0.795,0.824,-0.029,4
3,"1,007",0.510,0.824,-0.314,8
4,"1,009",0.862,0.744,0.118,4


To evaluate each surgeon’s performance, I needed to link each patient’s improvement score back to the surgeon who performed their operation.

Since each patient has exactly one associated surgeon, I:
- Selected `patient_id` and `surgeon_id` from the `scored_answers_sql` table
- Used `.drop_duplicates()` to make sure each patient appears only once
- Merged this into the `patient_scores_pivot` table on `patient_id`

I used `scored_answers_sql` because, it was built after filtering to hip surgeries, it guarantees we’re only merging in **surgeons related to hip replacements**, staying true to the scope of the analysis.

Each row in `patient_scores_pivot` now includes:
- Pre- and post-surgery health scores
- Improvement score
- The `surgeon_id` responsible for the operation



### Step 8: Check for Incomplete Patient Records

Before calculating surgeon performance, we need to confirm that each patient has both a pre- and post-surgery score.

Patients with missing scores will have `NaN` in the `pre_score`, `post_score`, or `improvement` columns. These rows need to be excluded because we can't calculate improvement from only one side.

Below, I check how many such rows exist before filtering.


In [81]:
# Total number of patients before filtering
total_patients = patient_scores_pivot.shape[0]

# Number of complete (valid) patients
valid_patients = patient_scores_pivot.dropna(subset=['pre_score', 'post_score', 'improvement']).shape[0]

# Number of incomplete patients
incomplete_patients = total_patients - valid_patients

# Display
print(f"Total patients: {total_patients}")
print(f"Complete (valid) patients: {valid_patients}")
print(f"Incomplete patients removed: {incomplete_patients}")
print(f"% Dropped: {round((incomplete_patients / total_patients) * 100, 2)}%")


Total patients: 12359
Complete (valid) patients: 9552
Incomplete patients removed: 2807
% Dropped: 22.71%


Out of `12,359` total patients, `2,807` were removed from analysis due to missing pre- or post-surgery scores.

This left `9,552` patients with complete data, which is a large and reliable sample for calculating health improvement and evaluating surgeon performance.

Only patients with both pre- and post-surgery responses were included to ensure fair and accurate comparisons.


### Step 9: Filter Valid Patients and Rank Surgeons

Some patients may have only filled out the pre- or post-surgery survey, but not both. These will show up as missing (`NaN`) values in `pre_score`, `post_score`, or `improvement`.

Since we can’t compute meaningful improvement without both scores, we drop those rows.


In [82]:
# Step 1: Drop patients with missing pre/post/improvement
valid_scores = patient_scores_pivot.dropna(subset=['pre_score', 'post_score', 'improvement'])

Then, we group by `surgeon_id` and calculate:
- Average improvement per surgeon
- Total number of patients treated
- Standard deviation to measure consistency

This gives us a performance snapshot of each surgeon.

In [83]:
# Step 2: Group by surgeon and calculate stats
surgeon_summary = valid_scores.groupby('surgeon_id').agg(
    avg_improvement=('improvement', 'mean'),
    num_patients=('patient_id', 'count'),
    std_improvement=('improvement', 'std')
).reset_index()

# Step 3: Merge with surgeon names
surgeon_summary = surgeon_summary.merge(
    surgeons.rename(columns={'id': 'surgeon_id', 'name': 'surgeon_name'}),
    on='surgeon_id',
    how='left'
)

# Step 4: Sort by improvement
surgeon_summary = surgeon_summary.sort_values(by='avg_improvement', ascending=False)

# Preview
surgeon_summary.head()


,surgeon_id,avg_improvement,num_patients,std_improvement,surgeon_name
4,5,0.121311,973,0.253740,Obi-Wan Kenobi
8,9,0.120326,958,0.255119,Luke Skywalker
6,7,0.120231,954,0.263165,Mon Mothma
2,3,0.117801,944,0.260637,Yoda
5,6,0.113151,963,0.257487,Darth Vader




-> The `avg_improvement` column represents the average change in health score across all patients treated by a surgeon — higher values indicate better overall outcomes.


-> The `std_improvement` column shows how consistent each surgeon's outcomes are — lower values indicate more consistent (less variable) patient improvement, while higher values suggest greater variability.


Results: 

- Merged in the surgeon names using the `surgeons` table. I kept the names exactly as provided — no titles like "Dr." were included in the original data, so I've preserved that format.

- Sorted the results by `avg_improvement` in descending order to highlight the highest-performing surgeons first.

- The above result gives us a clean, aggregated view of surgeon performance based on real patient outcomes.


### Step 10: Calculating weighted score

To rank surgeons more fairly, I calculated a **weighted score** that considers both:

- **How effective** a surgeon is (average improvement)
- **How experienced** they are (number of patients treated)

The formula used is:

> weighted_score = avg_improvement × log(1 + num_patients)


This ensures that surgeons who consistently deliver strong outcomes at scale are ranked higher, while avoiding overvaluing small sample sizes. The log function gives diminishing returns for volume, so large patient counts don’t overpower the improvement metric.

In [84]:
#Add Weighted Score (Effectiveness × Experience)
import numpy as np

surgeon_summary['weighted_score'] = (
    surgeon_summary['avg_improvement'] * np.log1p(surgeon_summary['num_patients'])
)


SELECT
  surgeon_id,
  AVG(post_score - pre_score) AS avg_improvement,
  COUNT(*) AS num_patients,
  STDDEV(post_score - pre_score) AS std_improvement
FROM patient_scores_pivot
WHERE pre_score IS NOT NULL AND post_score IS NOT NULL
GROUP BY surgeon_id 

*Note:* This is a conceptual SQL equivalent. In practice, I used Pandas to pivot and calculate improvements, since `pandasql` (SQLite) does not support `PIVOT` or `STDDEV()` functions.


Now that we have patient-level improvements, we aggregate results at the surgeon level.

For each `surgeon_id`, I calculated:
- `avg_improvement`: Average improvement across their patients
- `std_improvement`: Standard deviation (consistency of outcomes)
- `num_patients`: Number of patients treated
- `weighted_score`: A composite metric = avg_improvement × log(num_patients), which balances effectiveness and experience

This final table gives a ranked snapshot of surgeon performance across all hip replacement surgeries.


### Step 11: Final Surgeon Performance Table

This table brings together all the key metrics used to evaluate each surgeon’s performance based on patient outcomes after hip replacement surgeries.

**Step 1:** I grouped the cleaned patient dataset by `surgeon_id` to calculate:
- `avg_improvement`: Average improvement across all their patients
- `std_improvement`: Standard deviation of those improvements (how consistent they are)
- `num_patients`: Number of hip replacement patients treated

**Step 2:** I merged in the surgeon names from the `surgeons` table.

**Step 3:** I added a `weighted_score` to fairly rank surgeons by combining skill and experience:
> weighted_score = avg_improvement × log(1 + num_patients)

This gives a boost to surgeons who performed well **and** treated more patients — without letting volume overpower quality.

**Step 4:** I sorted the table by `weighted_score` in descending order to highlight the top performers.


Note: While I calculated some of these metrics earlier for exploratory purposes, this block builds the final, clean version of the surgeon performance table, with all scoring metrics combined and sorted by `weighted_score` for reporting.


In [85]:
import numpy as np

# Step 1: Group by surgeon_id and calculate key metrics
surgeon_scores = valid_scores.groupby('surgeon_id').agg(
    avg_improvement=('improvement', 'mean'),
    std_improvement=('improvement', 'std'),
    num_patients=('patient_id', 'count')
).reset_index()

# Step 2: Merge in surgeon names
surgeon_scores = surgeon_scores.merge(
    surgeons.rename(columns={'id': 'surgeon_id', 'name': 'surgeon_name'}),
    on='surgeon_id',
    how='left'
)

# Step 3: Add weighted score = avg improvement × log(num_patients)
surgeon_scores['weighted_score'] = (
    surgeon_scores['avg_improvement'] * np.log1p(surgeon_scores['num_patients'])
)

# Step 4: Sort by weighted_score (or avg_improvement)
surgeon_scores = surgeon_scores.sort_values(by='weighted_score', ascending=False)

# Preview the final results
surgeon_scores.head()


,surgeon_id,avg_improvement,std_improvement,num_patients,surgeon_name,weighted_score
4,5,0.121311,0.253740,973,Obi-Wan Kenobi,0.834794
8,9,0.120326,0.255119,958,Luke Skywalker,0.826143
6,7,0.120231,0.263165,954,Mon Mothma,0.824988
2,3,0.117801,0.260637,944,Yoda,0.807075
5,6,0.113151,0.257487,963,Darth Vader,0.777468


This table is the final result used to identify the **most and least skillful surgeons** based on patient-reported improvements.

# Visualization

While the technical analysis helps validate our logic, **stakeholders typically care most about the outcome** , who is performing well, who isn't, and what should be done next.

These visualizations are designed to:
- Highlight clear rankings (best vs. worst surgeons)
- Surface tradeoffs between experience and effectiveness
- Reveal patterns or inconsistencies that may require follow-up

This is the kind of output I would share in a presentation or report to a clinical lead, operations manager, or business stakeholder. 

For all visualizations in this section, I used **Plotly**, a Python library that supports interactive, stakeholder-friendly charts ideal for exploring and presenting analytical results

---

### Bar Chart: Top 10 Surgeons by Average Improvement

This chart shows the top 10 surgeons based on average patient improvement after hip replacement surgery. Higher bars represent greater average outcome improvements.


In [86]:
import plotly.express as px

# Top 10 by average patient improvement
top10 = surgeon_scores.sort_values(by='avg_improvement', ascending=False).head(10)

fig = px.bar(
    top10,
    x='avg_improvement',
    y='surgeon_name',
    orientation='h',
    color='avg_improvement',
    color_continuous_scale='Tealgrn',
    title='Top 10 Surgeons by Average Patient Improvement',
    labels={'avg_improvement': 'Average Improvement', 'surgeon_name': 'Surgeon'}
)

fig.update_layout(yaxis=dict(autorange='reversed'))
fig.show()


#### Interpretation: 

This chart ranks surgeons based on the **average improvement** reported by their patients after hip replacement surgery.

- **Obi-Wan Kenobi**, 
-  **Luke Skywalker**, and 
- **Mon Mothma** stand out as the top performers based on this metric.

This provides a clear, high-level view of effectiveness, but it doesn’t account for how many patients each surgeon treated or how consistent their results were. That’s where the next visualizations like the bubble and box plots, provide more context.

---


### Scatter Plot: Patient Volume vs. Average Improvement

This chart visualizes the relationship between a surgeon’s **effectiveness** (average improvement) and **experience** (number of patients treated).  
Each point represents a surgeon, and while marker size reflects outcome variability (standard deviation), the values are relatively similar, so bubble size does not vary significantly.

In [87]:
fig = px.scatter(
    surgeon_scores,
    x='num_patients',
    y='avg_improvement',
    size='std_improvement',
    color='weighted_score',
    hover_name='surgeon_name',
    color_continuous_scale='Viridis',
    size_max=40,
    title='Surgeon Performance: Patient Volume vs. Average Improvement'
)

fig.update_layout(
    xaxis_title='Number of Patients',
    yaxis_title='Average Improvement',
    legend_title='Weighted Score'
)

fig.show()


#### Interpretation: 

- This chart highlights which surgeons deliver strong results **at scale** 

- Surgeons in the **top-right quadrant** like **Obi-Wan Kenobi** and **Luke Skywalker** performed well across a large number of patients, showing strong, scalable effectiveness.

- Since most surgeons had similar outcome consistency (standard deviation), the **bubble size is visually less impactful** here but it’s included for completeness.

---


### Box Plot: Distribution of Patient Improvement per Surgeon

This box plot shows the **spread of improvement scores** for the top 6 surgeons (by patient volume). 

Each box represents the **distribution of patient improvements** for a single surgeon, allowing us to assess not just their average performance, but also their **consistency**.

- The **median line** shows the typical improvement for each surgeon.
- The **box height** represents the interquartile range (IQR), the middle 50% of outcomes.
- **Whiskers and outliers** show the range and variability.

A narrow box with fewer outliers suggests **consistent performance**, while a wide box or high spread indicates more **variation** in patient outcomes.

This visualization helps identify surgeons who not only perform well on average, but also deliver stable results across many patients.


In [88]:
# Pick top 6 surgeons by volume or average improvement
top_surgeons = surgeon_scores.sort_values(by='num_patients', ascending=False).head(6)['surgeon_id']

box_data = valid_scores[valid_scores['surgeon_id'].isin(top_surgeons)]

# Merge surgeon names
box_data = box_data.merge(
    surgeons.rename(columns={'id': 'surgeon_id', 'name': 'surgeon_name'}),
    on='surgeon_id',
    how='left'
)

fig = px.box(
    box_data,
    x='surgeon_name',
    y='improvement',
    title='Improvement Distribution per Surgeon (Top 6)',
    labels={'surgeon_name': 'Surgeon', 'improvement': 'Improvement Score'},
    color='surgeon_name'
)

fig.update_layout(showlegend=False)
fig.show()


#### Interpretation: 

This box plot shows how improvement scores vary for each of the top 6 surgeons.

What to look for:

- The **box height** (IQR) shows the middle 50% of patient outcomes.
- The **line in the middle** is the median (typical outcome).
- **Dots** are outliers — patients who had unusually high or low changes in health.

Surgeon-by-surgeon observations:

- **Boba Fett**  
  - Wide spread with many negative outliers  
  - Median slightly above zero, but variability is high  
  → Results are inconsistent

- **Jar Jar Binks**  
  - Similar spread to Boba Fett, with low median  
  - Several outliers below -0.5  
  → Results lean negative and highly variable

- **Darth Vader**  
  - More centered around positive outcomes  
  - Smaller IQR and fewer extreme outliers  
  → More stable performance

- **Obi-Wan Kenobi**  
  - Moderate spread, median well above zero  
  - One or two lower outliers  
  → Consistently strong outcomes with a few exceptions

- **Padme Amidala**  
  - Lower median and wider spread  
  - More patients saw limited or even negative improvement  
  → Underperformance with significant variability

- **Luke Skywalker**  
  - High median and relatively tight IQR  
  - Lower whisker extends, but outcomes are mostly positive  
  → Strong and consistent performer


Takeaway:
While Obi-Wan and Luke both ranked high in earlier charts, this box plot shows **Luke Skywalker** had slightly more consistent outcomes, while **Jar Jar Binks** and **Padme Amidala** had wide variability and more patients with poor recovery.


---


### Correlation Heatmap: Surgeon Performance Metrics

This heatmap shows how the key performance metrics relate to one another:

- **Average Improvement**: The mean improvement across all patients for each surgeon  
- **Standard Deviation**: Variability in outcomes, higher means less consistency  
- **Number of Patients**: Total patients treated  
- **Weighted Score**: A combined metric of effectiveness × experience (`avg_improvement × log(1 + num_patients)`)

How to read It:
- **Values range from -1 to +1**:
  - `+1` means perfect positive correlation
  - `-1` means perfect negative correlation
  - `0` means no linear relationship
- **Darker shades** indicate stronger relationships.


In [89]:
import plotly.figure_factory as ff

# Correlation matrix
corr_matrix = surgeon_scores[['avg_improvement', 'std_improvement', 'num_patients', 'weighted_score']].corr()

# Create heatmap
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=[[f'{val:.2f}' for val in row] for row in corr_matrix.values],
    colorscale='Viridis',
    showscale=True
)

fig.update_layout(title='Correlation Heatmap: Surgeon Performance Metrics')
fig.show()


#### Interpretation: 

- **`avg_improvement` and `weighted_score`** are strongly correlated as expected, since the weighted score includes improvement directly.
- **`std_improvement` and `avg_improvement`** are negatively correlated (`-0.33`), meaning surgeons with higher average outcomes tend to have more consistent results.
- **`num_patients` and `weighted_score`** show a mild positive correlation, which validates the use of a log scale, rewarding experience without letting it dominate.
- **`num_patients` and `avg_improvement`** are only weakly correlated, showing that simply treating more patients doesn’t guarantee better outcomes.

This chart helps confirm that the metrics used in the final scoring (especially `weighted_score`) provide **complementary information**, not redundancy. It supports the fairness and structure of the final surgeon ranking.

---

### Bar Chart: Patient Volume per Surgeon

In [90]:
fig = px.bar(
    surgeon_scores.sort_values(by='num_patients'),
    x='surgeon_name',
    y='num_patients',
    title='Patient Volume per Surgeon'
)
fig.show()

#### Interpretation: 

The Patient Volume per Surgeon bar chart shows that:

- All surgeons treated a fairly similar number of patients (roughly 900–990).

- There is no huge imbalance where one surgeon saw 10× more cases than another.

- This means the log scaling in the weighted score didn’t disproportionately advantage or penalize anyone.

--- 

### Bar Chart: Final Ranking of surgeons by weighted score

In [91]:
import plotly.express as px

# Sort all surgeons by weighted_score
fig = px.bar(
    surgeon_scores.sort_values(by='weighted_score', ascending=False),
    x='weighted_score',
    y='surgeon_name',
    orientation='h',
    color='avg_improvement',
    color_continuous_scale='Blues',
    title='Final Ranking of Surgeons by Weighted Score',
    labels={
        'weighted_score': 'Weighted Score (Effectiveness × Experience)',
        'surgeon_name': 'Surgeon',
        'avg_improvement': 'Avg Improvement'
    }
)

fig.update_layout(yaxis=dict(autorange='reversed'))
fig.show()


#### Interpretation: 

This chart presents the final ranking of surgeons based on their **weighted score**, which balances average patient improvement with the number of patients treated.

- **Obi-Wan Kenobi**, **Luke Skywalker**, and **Mon Mothma** lead the rankings, consistently delivering strong outcomes across a high volume of patients.
- Surgeons like **Padme Amidala** and **Princess Leia** rank lower, with lower average improvement and fewer patients treated.
- The color gradient represents average improvement, reinforcing that top-ranked surgeons not only had scale, but also strong effectiveness.

This view provides a fair and balanced summary of surgeon performance, rewarding both skill and scale

---

### Performance Metrics Table : Top 3 vs. Bottom 3 Surgeons

In [92]:
# performance metrics of the top 3 and bottom 3 surgeons.
pd.concat([
    surgeon_scores.head(3),
    surgeon_scores.tail(3)
])

,surgeon_id,avg_improvement,std_improvement,num_patients,surgeon_name,weighted_score
4,5,0.121311,0.253740,973,Obi-Wan Kenobi,0.834794
8,9,0.120326,0.255119,958,Luke Skywalker,0.826143
6,7,0.120231,0.263165,954,Mon Mothma,0.824988
1,2,0.110510,0.261613,938,Darth Maul,0.756418
0,1,0.106901,0.257088,905,Princess Leia,0.727890
9,10,0.104854,0.261343,986,Padme Amidala,0.722933


In [93]:
surgeon_scores.to_csv("surgeon_performance_summary.csv", index=False)


I have exported the final surgeon ranking table so it can be shared or used in future dashboards.


# Conclusion

> Who are the most and least skilful surgeons for hip replacement operations?
 
This analysis set out to identify the most and least skillful surgeons based on patient-reported outcomes after hip replacement surgeries. Using the EQ-5D-5L survey data, I calculated pre- and post-surgery health scores for each patient, measured individual improvement, and aggregated performance metrics at the surgeon level.

Here’s a summary of what the analysis and visualizations revealed:

----

###  Top Performers (Final Weighted Score)

- **Obi-Wan Kenobi**, 
- **Luke Skywalker**, and 
- **Mon Mothma** 

consistently delivered the best outcomes. These surgeons showed both high average patient improvement and treated large patient volumes, making their performance both **effective and scalable**.

----

###  Underperformers

- **Padme Amidala**, 
- **Princess Leia**, and 
- **Darth Maul** 

ranked at the bottom based on lower average improvement and weaker overall consistency.

----

### Visual Insights Recap

- **Bar Chart – Average Improvement:**  
  Clearly shows who helped patients improve the most, but doesn’t account for volume or consistency.

- **Bubble Chart – Volume vs. Effectiveness:**  
  Showed which surgeons performed well at scale. Obi-Wan and Luke stood out as strong and experienced. Bubble size (standard deviation) was included but showed limited visual difference.

- **Box Plot – Improvement Distribution:**  
  Revealed that **Luke Skywalker** and **Darth Vader** delivered more consistent results. **Jar Jar Binks** and **Padme Amidala** showed wide variability, including negative outcomes.

- **Correlation Heatmap:**  
  Validated that the weighted score balances effectiveness and experience without duplicating effort. It also confirmed that more improvement tends to come with more consistency (negative correlation with std dev).

- **Final Ranking Chart (Weighted Score):**  
  Provided a fair, balanced view of surgeon performance by combining average improvement and patient volume using a log-weighted score.

- **Top vs. Bottom 3 Table:**  
  Directly compared high- and low-performing surgeons across all key metrics, reinforcing the performance gap.

----

### Recommendations 

If this were a real clinical dashboard or internal report, here are a few next steps I would recommend:

- **Track performance over time**: Add timestamps to monitor surgeon performance trends (e.g., monthly improvement).
- **Add complexity adjustment**: Factor in patient condition at baseline to make comparisons even fairer.
- **Highlight at-risk patients**: Flag patients with no improvement or negative scores for follow-up.

These additions could help hospital teams monitor outcomes in real-time and take early action when performance drops.

This analysis doesn't just rank surgeons, it helps explain **why** some perform better than others, and how outcome data can be used to measure both skill and consistency in a clinical context. These insights could easily feed into performance reviews, clinical training programs, or operational dashboards.